# 문제 정의

제공 받은 Dataset은 age, anaemia, high blood pressure, diabetes 등의 병원데이터라고 추측되는 데이터이고 DEATH_EVENT는 해당 환자의 사후관리 기간 중 사망한 경우를 나타냅니다.

라이브러리 임포트

In [1]:
# Libraies for reading and handling the data
import numpy as np
import pandas as pd
import re

# Libraries for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Libraries for data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

# Libraries for creating ML model
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Library for Analysing the ML model
from sklearn import metrics

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Selecting previously unselected package fonts-nanum.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


데이터 준비하기

In [3]:
# csv 파일을 데이터프레임으로 변환
df = pd.read_csv('data.csv', header=None)

In [4]:
# 열 이름 지정
df.columns = ['age','anamia','creatinine_phosphokinase','diabetes','ejection_fraction',
              'high_blood_pressure','platelets','serum_creatinine','serum_sodium',
              'sex','smoking','time','DEATH_EVENT']

데이터 탐색 및 데이터 전처리

In [5]:
# column 추가로 중복된 첫 번째행 삭제
df= df.drop(0,0)
df.head()

,age,anamia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
1,75,0,582,0,20,1,265000,1.9,130,1,0,4,1
2,55,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
3,65,0,146,0,20,0,162000,1.3,129,1,1,7,1
4,50,1,111,0,20,0,210000,1.9,137,1,0,7,1
5,65,1,160,1,20,0,327000,2.7,116,0,0,8,1


In [6]:
# age 열의 자료형 변경 (문자열 -> 숫자)
df['age'].unique()

array(['75', '55', '65', '50', '90', '60', '80', '62', '45', '49', '82',
       '87', '70', '48', '68', '53', '95', '58', '94', '85', '69', '72',
       '51', '57', '42', '41', '67', '79', '59', '44', '63', '86', '66',
       '43', '46', '61', '81', '52', '64', '40', '60.667', '73', '77',
       '78', '54', '47', '56'], dtype=object)

In [7]:
# 60.667를 np.nan으로 변경
df['age'].replace('60.667',np.nan,inplace=True)
df.info

<bound method DataFrame.info of     age anamia creatinine_phosphokinase diabetes  ... sex smoking time DEATH_EVENT
1    75      0                      582        0  ...   1       0    4           1
2    55      0                     7861        0  ...   1       0    6           1
3    65      0                      146        0  ...   1       1    7           1
4    50      1                      111        0  ...   1       0    7           1
5    65      1                      160        1  ...   0       0    8           1
..   ..    ...                      ...      ...  ...  ..     ...  ...         ...
295  62      0                       61        1  ...   1       1  270           0
296  55      0                     1820        0  ...   0       0  271           0
297  45      0                     2060        1  ...   0       0  278           0
298  45      0                     2413        0  ...   1       1  280           0
299  50      0                      196        0  ...  

In [8]:
# nan 확인
df['age'].unique()

array(['75', '55', '65', '50', '90', '60', '80', '62', '45', '49', '82',
       '87', '70', '48', '68', '53', '95', '58', '94', '85', '69', '72',
       '51', '57', '42', '41', '67', '79', '59', '44', '63', '86', '66',
       '43', '46', '61', '81', '52', '64', '40', nan, '73', '77', '78',
       '54', '47', '56'], dtype=object)

In [9]:
# 데이터안 nan값 60으로 변경
df=df.fillna(60)

In [10]:
# 변경 사항 확인
df['age'].unique()

array(['75', '55', '65', '50', '90', '60', '80', '62', '45', '49', '82',
       '87', '70', '48', '68', '53', '95', '58', '94', '85', '69', '72',
       '51', '57', '42', '41', '67', '79', '59', '44', '63', '86', '66',
       '43', '46', '61', '81', '52', '64', '40', 60, '73', '77', '78',
       '54', '47', '56'], dtype=object)

In [11]:
# 열들의 자료형 변경 (문자열 -> 숫자)
df['age']=df['age'].astype('int')

df['creatinine_phosphokinase']=df['creatinine_phosphokinase'].astype('int')

df['ejection_fraction']=df['ejection_fraction'].astype('int')

df['platelets']=df['platelets'].astype('float')

df['serum_creatinine']=df['serum_creatinine'].astype('float')

df['serum_sodium']=df['serum_sodium'].astype('int')

df['time']=df['time'].astype('int')

df['DEATH_EVENT']=df['DEATH_EVENT'].astype('int')

In [12]:
# 숫자 형태의 변수 -> 카테고리 형태로 변경
categorical_var=['anamia','diabetes','high_blood_pressure','sex','smoking']
df[categorical_var] = df[categorical_var].astype('category')

형변환 완료한 데이터 형식

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 1 to 299
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   age                       299 non-null    int64   
 1   anamia                    299 non-null    category
 2   creatinine_phosphokinase  299 non-null    int64   
 3   diabetes                  299 non-null    category
 4   ejection_fraction         299 non-null    int64   
 5   high_blood_pressure       299 non-null    category
 6   platelets                 299 non-null    float64 
 7   serum_creatinine          299 non-null    float64 
 8   serum_sodium              299 non-null    int64   
 9   sex                       299 non-null    category
 10  smoking                   299 non-null    category
 11  time                      299 non-null    int64   
 12  DEATH_EVENT               299 non-null    int64   
dtypes: category(5), float64(2), int64(6)
memory usage:

In [14]:
df.head()

,age,anamia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
1,75,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
2,55,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
3,65,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
4,50,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
5,65,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


데이터 분리하기

In [15]:
# X(시험지), y(정답지) 분리하기
X = df.iloc[:,:-1]
y = df['DEATH_EVENT']

In [16]:
# 원-핫 인코딩 처리
# 컬럼 -> 원-핫 인코딩 처리
temp = pd.get_dummies(X[categorical_var], drop_first=True)

In [17]:
# 기존 데이터랑 합치기
X_modified = pd.concat([X,temp], axis=1)

In [18]:
# 기존 컬럼 삭제
X_modified.drop(categorical_var, axis=1, inplace=True)

In [19]:
X_modified.head()

,age,creatinine_phosphokinase,ejection_fraction,platelets,serum_creatinine,serum_sodium,time,anamia_1,diabetes_1,high_blood_pressure_1,sex_1,smoking_1
1,75,582,20,265000.00,1.9,130,4,0,0,1,1,0
2,55,7861,38,263358.03,1.1,136,6,0,0,0,1,0
3,65,146,20,162000.00,1.3,129,7,0,0,0,1,1
4,50,111,20,210000.00,1.9,137,7,1,0,0,1,0
5,65,160,20,327000.00,2.7,116,8,1,1,0,0,0


훈련용, 테스트 데이터 분리하기

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_modified,y,
                                                    train_size=0.7, # test_size
                                                    random_state=7)

BAG 분류 모형 - sklearn 사용

In [55]:
# sklearn 라이브러리에서 BAG 분류 모형 가져오기
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier

bag = BaggingClassifier(LogisticRegression(), n_estimators=100, oob_score=True, n_jobs=-1, random_state=7)

In [56]:
# 훈련 데이터를 사용하여 모형 학습
bag.fit(X_train, y_train)

BaggingClassifier(base_estimator=LogisticRegression(C=1.0, class_weight=None,
                                                    dual=False,
                                                    fit_intercept=True,
                                                    intercept_scaling=1,
                                                    l1_ratio=None, max_iter=100,
                                                    multi_class='auto',
                                                    n_jobs=None, penalty='l2',
                                                    random_state=None,
                                                    solver='lbfgs', tol=0.0001,
                                                    verbose=0,
                                                    warm_start=False),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=100, n_jobs=-1, oob_score=True,
                  random_state=7, verb

모델 성능 평가

In [57]:
# BAG 모델 성능 평가
from sklearn import metrics

print('테스트 세트 정확도: {:.2f}%'.format(bag.score(X_test, y_test)*100))
print('OOB 샘플의 정확도: {:.3f}'.format(bag.oob_score_))

테스트 세트 정확도: 82.22%
OOB 샘플의 정확도: 0.804
